#◢ Setup

In [238]:
import sys
import os
import json
from itertools import groupby
from operator import itemgetter
from typing import List, Tuple, Dict

import gin
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

sys.path.append('..')

from vehicle_nowcasting.data.data_loader import build_source_from_metadata, make_dataset, generate_anchors
from vehicle_nowcasting.data.generate_anchors import generate_anchors as gen_anch
from vehicle_nowcasting.utils import iou, plot_bb, change_box_order

%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


##◢ Load metadata

In [239]:
main_dir = '..'
data_dir = os.path.join(main_dir, 'data')
metadata = pd.read_csv('../data/metadata.csv')
label_map = json.load(open('../data/label_map.json', 'r'))

In [240]:
sources = build_source_from_metadata(metadata,
                                     label_map,
                                     data_dir)

In [241]:
ds = make_dataset(sources)

In [266]:
anch = tf.constant(gen_anch(base_size=4,
                               ratios=[1, 1.25, 1.5, 1.75, 2, 2.25, 2.5],
                               scales=2 ** np.arange(3, 10)))
#anchors = change_box_order(anchors, with_classes=False)

In [199]:
bbox = next(iter(ds))[1][:, 1:]
bbox_ = change_box_order(tf.cast(bbox[:, 1:], tf.float64), with_classes=False)

In [250]:
anch = change_box_order(anch, with_classes=False)
anchors = change_box_order(anchors, with_classes=False)
#anchors = tf.reshape(anchors[:, -2:], (1, 2, -1))

In [209]:
foo = tf.reshape(anchors[:, 2:] - anchors[:, :2], (1, 2, -1))

In [214]:
var = tf.reshape(bbox[:, 2:] - bbox[:, :2], (-1, 2, 1))
var = tf.cast(var, tf.float64)

In [227]:
cond = (foo - var < EPS)
#cond = cond[:, 0, :] == cond[:, 1, :]